# Accessing and creating content

Your GIS can host a [variety of maps, web layers, analytical tools, apps](http://doc.arcgis.com/en/arcgis-online/reference/geo-info.htm) and [individual files](http://doc.arcgis.com/en/arcgis-online/share-maps/supported-items.htm). Using the `gis` module, you can search for, access and manage all your folders, contents and automate such maintenance workflows as scripts.

## What is Content?

- Digital information
- Excel spreadsheets, spatial data, etc..
- Services
- Web Maps
- Imagery and Raster Data

## What a User Sees

<img src="./img/sample_item.png" />

## What a Programmer Sees


In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')
item = gis.content.get("ab399b847323487dba26809bf11ea91a")
print(item)

In [ ]:
item.__dict__

## Import libraries

In [ ]:
import os
import datetime as dt
import pandas 
from arcgis.gis import GIS

## About search

As mentioned in the [`gis module`](https://developers.arcgis.com/python/guide/the-gis-module/) guide, the Python API uses [Resource Manager classes](https://developers.arcgis.com/python/guide/the-gis-module/) to manage Web GIS users, groups, datastores, and content. You access the [`UserManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager), [`GroupManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#groupmanager) and [`ContentManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager) classes as the `users`, `groups`, and `content` properties of the [`GIS`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis), respectively. (See [`Helper objects`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis) for details.)

Each resource manager implements the [ArcGIS REST API](https://developers.arcgis.com/rest/users-groups-and-items/search.htm) `search` operation as a method. For example, `gis.content.search()`. It's important to understand that the `search` mechanism uses many different inputs to find possible matches, ranks them and returns appropriate results. The `search` becomes ideal for human interaction, but `fuzzy` when looking for specific records programmatically. The search results are non-deterministic. Using `search` may not necessarily be the best approach for finding specific items, but more a group of items from which to further filter. 

It's also important to know that using `search` programmatically, like with the Python API, does not correspond identically to searching with an application written in a different language.  The various `search` options in the ArcGIS Online or Portal for ArcGIS Enterprise interfaces work differently than the Python API resource managers' `search`. Different applications may use different inputs. The relationship between a content `search` in one application, like a Portal or ArcGIS Online viewer, is not one-to-one with a content search using the Python API even when logged in as the same user.

Let's look at an example to clarify:

In [ ]:
# Search for Feature Layers owned by the logged-in user
my_content = gis.content.search(query="owner:" + gis.users.me.username, 
                                item_type="Feature Layer", 
                                max_items=15)

my_content

Lets see what you get when you access `content` property

## Creating new content

To create new items on your GIS, you use the [`add()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.add) method on a `ContentManager` instance. Again, rather than creating an instance of the `ContentManager` directly, the Python API implments the class as the `content` property of your `GIS` object.

The `add()` method accepts a dictionary containing the properties of the item to be created. The optional `data` parameter accepts a string path to a file. You can use the optional `metadata` parameter to specify an XML file containing metadata information.

All content you add this way is added to the authenticated user's contents. Let us connect to our Enterprise instance, add a csv file as an item and then publish a feature layer from it.

For example, set up a path to the data:
`data_path = os.path.join(r"<your_drive>:\\", "path", "to", "data")`

In [ ]:
data_path = (r".\data")

In [ ]:
csv_path = os.path.join(data_path, "earthquakes.csv")
csv_properties={'title':'Earthquakes around the world from 1800s to early 1900s',
                'description':'Measurements from globally distributed seismometers',
                'tags':'arcgis, python, earthquake, natural disaster, emergency'}
thumbnail_path = os.path.join(data_path, "earthquake.png")

earthquake_csv_item = gis.content.add(item_properties=csv_properties, data=csv_path,
                                     thumbnail = thumbnail_path)

The `add()` method returns an object of type `arcgis.gis.Item` representing the added item. In Jupyter notebook environment, you can visualize it by querying the item

In [ ]:
earthquake_csv_item

**Note**: You can specify most properties of an item as a dictionary to the `item_properties` parameter. Refer to the [API ref doc](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.add) of this method for a definitive list. You could also specify an empty dictionary to this parameter, the downside is search cannot index these items efficiently if it does not have sufficient metadata information, hence that is not a good programming practice.

### Publishing an item as a web layer
In the example above you added a csv [`Item`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item) to the portal. Items such as csv files, service definitions, file geodatabases, shapefiles and packages such as tile, scene and vector tile packages can be published into hosted web layers. You call the [`publish()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.publish) method on the `Item` object to create the web layer.

Optionally, The `publish()` method accepts an optional `publish_parameters` dictionary where you can specify certain parameters. See the [publish REST API](http://resources.arcgis.com/en/help/arcgis-rest-api/index.html#/Publish_Item/02r300000080000000/) documentation for detailed descriptions on these parameters. For instance, the `address_fields` parameter allows you to specify which field in your csv or dataset contains address fields suitable for geocoding.

Let us publish the csv into a web feature layer. This csv contains latitude and longitude coordinates of each point, hence the `address_fields` parameter can be ignored. The `publish()` method returns an `Item` object corresponding to the web layer created.

In [ ]:
earthquake_feature_layer_item = earthquake_csv_item.publish()

In [ ]:
earthquake_feature_layer_item

Notice that the GIS used the same metadata (thumbnail, title, description) of the source item used for publishing. This helps provide context on what the new item is and also as a good starting point for the metadata which you can edit later.

<a id="importing-data-from-a-pandas-data-frame"></a>
### Importing data from a pandas data frame

[Pandas](http://pandas.pydata.org) is a popular data analysis library for Python. The data frame object of pandas allows you to store and analyze tabular information with ease.

Let us create a new pandas data frame representing locations of a few coastal cities read from a csv file. Note, in the example below, you could publish the csv as an item as shown earlier, but we will observe how you can filter that data through pandas and create a Spatially Enabled dataframe instead

In [24]:
# read csv as a pandas dataframe
ports_df = pandas.read_csv(os.path.join(data_path, "ports.csv"))
ports_df

,country,globalid,harborsize,label_position,latitude,longitude,port_name,short_form,geometry.x,geometry.y
0,US,{14FB62C9-1CEA-4F12-ACA4-14BB72F042BF},L,SW,37.816667,-122.416667,SAN FRANCISCO,SFO,-13627360,4553559
1,US,{6EE30C76-399D-42F5-B6ED-403C94DFFCD4},S,SW,37.783333,-122.266667,ALAMEDA,NGZ,-13610660,4548863
2,US,{A6A21CC3-DE52-4B8A-A5B2-1CAA726E7F02},L,SW,37.816667,-122.333333,OAKLAND,OAK,-13618080,45535590
3,US,{265AD7A6-4F59-4181-88DE-194F23F64E92},S,SW,37.916667,-122.366667,POINT RICHMOND,RIC,-13621800,4567660
4,US,{B15D545F-83DD-4D3E-9051-6D9F290A8E9E},S,SW,38.100000,-122.266667,MARE ISLAND,None,-13610660,4593562
5,US,{A97E8B59-1E25-416E-9206-4B59A869E2F4},S,SW,38.083333,-122.250000,SOUTH VALLEJO,None,-13608810,4591205
6,US,{8EB57C5B-100B-4C50-B6B1-4D6B9B1C7545},S,SW,37.950000,-121.300000,STOCKTON,SCK,-13503050,4572365
7,US,{3B7B126E-0B60-49AF-89AA-4AC05402CFEA},S,SW,38.583333,-121.500000,SACRAMENTO,SMF,-13525320,4662162
8,US,{20378B4B-3E9E-47D2-AF42-4F0A31D5CA66},S,SW,40.800000,-124.183333,EUREKA,ACV,-13824030,4982886
9,US,{782C11E0-B02D-45AB-8BAD-97C1B5817812},M,SW,43.400000,-124.283333,EMPIRE,None,-13835160,5373055


In [25]:
# find latitude of SFO
lat = ports_df.loc[ports_df.port_name == 'SAN FRANCISCO']['latitude']
lat

0    37.816667
Name: latitude, dtype: float64

In [26]:
# only select ports that are to the south of SFO
ports_south_of_SFO = ports_df.loc[ports_df.latitude < lat[0]]
ports_south_of_SFO

,country,globalid,harborsize,label_position,latitude,longitude,port_name,short_form,geometry.x,geometry.y
1,US,{6EE30C76-399D-42F5-B6ED-403C94DFFCD4},S,SW,37.783333,-122.266667,ALAMEDA,NGZ,-13610660,4548863
11,US,{A04935D7-314D-43D3-AA39-99E91FE93BD5},M,SW,32.716667,-117.183333,SAN DIEGO,SAN,-13044790,3857756
12,US,{45CCFAFA-CE13-4EEE-907A-5C53A486C434},M,SW,33.766667,-118.183333,LONG BEACH,LGH,-13156110,3997514
13,US,{ECA58B5E-ADBB-4392-A6F2-1F5476E69375},S,SW,36.616667,-121.883333,MONTEREY,MRY,-13567990,4385809
15,US,{B8AC37F8-5AA6-4D27-A401-7B35A8E4B9D3},S,SW,21.950000,-159.350000,NAWILIWILI BAY,None,-17738760,2505523
16,US,{6D718925-1CF2-4A45-9E3D-B4FBE215BA71},S,SW,21.900000,-159.583333,PORT ALLEN,None,-17764740,2499523
17,US,{50ECBCE3-137E-45BF-A42E-F0D423434067},M,SW,21.300000,-157.866667,HONOLULU,HNL,-17573640,2427687
18,US,{9168C51F-28FF-4885-AFD5-18A1607E3808},S,SW,20.033333,-155.833333,KAWAIHAE,None,-17347290,2276980
19,US,{A2A35C01-C1D4-4FD0-B74E-424E1A9DE20F},M,SW,19.733333,-155.066667,HILO,ITO,-17261940,2241467


Now that we have the desired data frame, let us transform it to a SeDF and publish it.

In [28]:
sedf = pandas.DataFrame.spatial.from_xy(ports_south_of_SFO,"longitude","latitude")
sedf

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\geo\_io\fileops.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SHAPE"] = GeoArray(ags_geom)


,country,globalid,harborsize,label_position,latitude,longitude,port_name,short_form,geometry.x,geometry.y,SHAPE
1,US,{6EE30C76-399D-42F5-B6ED-403C94DFFCD4},S,SW,37.783333,-122.266667,ALAMEDA,NGZ,-13610660,4548863,"{""spatialReference"": {""wkid"": 4326}, ""x"": -122..."
11,US,{A04935D7-314D-43D3-AA39-99E91FE93BD5},M,SW,32.716667,-117.183333,SAN DIEGO,SAN,-13044790,3857756,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
12,US,{45CCFAFA-CE13-4EEE-907A-5C53A486C434},M,SW,33.766667,-118.183333,LONG BEACH,LGH,-13156110,3997514,"{""spatialReference"": {""wkid"": 4326}, ""x"": -118..."
13,US,{ECA58B5E-ADBB-4392-A6F2-1F5476E69375},S,SW,36.616667,-121.883333,MONTEREY,MRY,-13567990,4385809,"{""spatialReference"": {""wkid"": 4326}, ""x"": -121..."
15,US,{B8AC37F8-5AA6-4D27-A401-7B35A8E4B9D3},S,SW,21.950000,-159.350000,NAWILIWILI BAY,None,-17738760,2505523,"{""spatialReference"": {""wkid"": 4326}, ""x"": -159..."
16,US,{6D718925-1CF2-4A45-9E3D-B4FBE215BA71},S,SW,21.900000,-159.583333,PORT ALLEN,None,-17764740,2499523,"{""spatialReference"": {""wkid"": 4326}, ""x"": -159..."
17,US,{50ECBCE3-137E-45BF-A42E-F0D423434067},M,SW,21.300000,-157.866667,HONOLULU,HNL,-17573640,2427687,"{""spatialReference"": {""wkid"": 4326}, ""x"": -157..."
18,US,{9168C51F-28FF-4885-AFD5-18A1607E3808},S,SW,20.033333,-155.833333,KAWAIHAE,None,-17347290,2276980,"{""spatialReference"": {""wkid"": 4326}, ""x"": -155..."
19,US,{A2A35C01-C1D4-4FD0-B74E-424E1A9DE20F},M,SW,19.733333,-155.066667,HILO,ITO,-17261940,2241467,"{""spatialReference"": {""wkid"": 4326}, ""x"": -155..."


In [30]:
len(sedf)

9

In [29]:
sedf.spatial.to_featurelayer("Ports south of SFO")

The operation was attempted on an empty geometry.


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


<Item title:"Ports south of SFO" type:Feature Layer Collection owner:cede_esrich>

<a id="organizing-content"></a>
## Organizing content
Over time, your GIS can get filled with content. You can create folders on your GIS, just like in your file system, and use it to organize.

<a id="using-folders-to-organize-your-content"></a>
### Using folders to organize your content
The `create_folder()` and `delete_folder()` methods available on the `ContentManager` class can be used to manage your folders. Once created, you can use the `move()` method to move content into and out of folders.

Let us create a folder called 'ports' and move the `ports_item` into it 

In [ ]:
# create new folder
gis.content.create_folder(folder='ports')

In [ ]:
# move the ports_item into this folder
ports_item.move(folder='ports')

You can move content out of folders to root by calling the `move()` method and specifying `/` as folder name

In [ ]:
# move back to root
ports_item.move(folder='/')

## Summary
Thus, in this guide you observed how the `gis` module can be used to create new content and search for existing ones. Head over to the guide on [managing-your-content](https://developers.arcgis.com/python/guide/managing-your-content) next. When combined into a script, the ArcGIS API for Python becomes a powerful tool to automate your GIS content creation and management tasks.